In [2]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

# LLM-Setup
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",  # anpassen, falls nötig
    model="gemma:7b",                       # z.B. gemma:7b, mistral-7b, llama3-70b
    temperature=0.7,
    top_p=0.9,
    top_k=20,
    repeat_penalty=1.2,
    presence_penalty=1.5
)

template_kids = PromptTemplate(
    input_variables=["frage"],
    template=(
        "Du bist ein freundlicher Assistent, der Kindern auf ihre Fragen antwortet.\n"
        "Erkläre Dinge in einfacher, kindgerechter Sprache, damit Kinder im Alter von 6 bis 10 Jahren sie gut verstehen können.\n"
        "Sei positiv, ermutigend und benutze kurze, klare Sätze.\n"
        "Wenn es passt, darfst du Beispiele oder kleine Vergleiche verwenden, damit es noch besser verständlich wird.\n\n"
        "Frage: {frage}\n"
        "Antwort:"
    )
)

print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")

# Chat-Schleife
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = llm.invoke(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print(f"⚠️ Fehler: {e}")


🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Saarbruck ist eine Stadt in Südwestdeutschland an der Grenze zu Frankreich.
🤖 Bot: Auf Wiedersehen!
